## Imports

In [ ]:
# Python standard library
import sys
import csv


# Scientific computing
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.ndimage
import scipy.interpolate
#import seaborn as sns


# Program specific
sys.path.append('/home/prestonh/Desktop/Research/pore_stats/pore_stats/rp/')
sys.path.append('/home/prestonh/Desktop/Research/pore_stats/pore_stats/oi/')
import resistive_pulse as rp
import rp_file
import optical_imaging as oi
import oi_file


# Jupyter
from IPython.display import HTML

## Load data

In [ ]:
date = '10-10-2017/'
particle = '293-T_1/'
channel = '10-20-10_0/'
file_index = '0/'

base_path = '/home/prestonh/Desktop/Research/cancer_cells/data/'

oi_vid_file_path = base_path + date + particle + channel + 'oi/bin/test_camera_' + file_index.replace('/','')
oi_events_file_path = base_path + date + particle + channel + 'oi/events/test_camera_' + file_index.replace('/','') + '_events.json'


res_x = 384
res_y = 112
fps = 250000.
exposure_time = .5*10**(-6.)

In [ ]:
# Load video
oi_vid = oi_file.Video(oi_vid_file_path, res_x, res_y, fps)

# Load events
oi_events = oi_file.open_event_file_json(oi_events_file_path)

In [ ]:
save_base_path = '/home/prestonh/Desktop/Research/cancer_cells/analysis/data/' + date + particle + channel + file_index

print save_base_path

In [ ]:
reload(oi)
template_index = 90
template_frame = oi_vid.get_frame(template_index)
c0 = [78, 80]
c1 = [78, 55]
c2 = [325, 53] 
c3 = [325, 75]


oi_stage = oi.Stage(template_frame, c0, c1, c2, c3)
oi_stage.plot_stage()

In [ ]:
oi_ellipse_base_path = '../data/'
oi_ellipse_path = oi_ellipse_base_path + date + particle + channel + file_index + 'ellipses'

ellipsess = []


with open(oi_ellipse_path, 'r') as file_handle:
    file_reader = csv.reader(file_handle, delimiter = '\t')
    
    # Skip header
    header_length = 2
    for i in range(header_length):
        next(file_reader)
        
        
    for row in file_reader:
        
        if row[0] == 'event #':
            # New event
            ellipsess.append([])
            continue
            
        else:
            ellipse = []
            for ele in row:
                try:
                    ellipse.append(float(ele))
                except:
                    ellipse.append(0)
                    
            ellipsess[-1].append(ellipse)

In [ ]:
# Load keep indices
filter_file_path = '../data/' + date + particle + channel + file_index + 'filter'

with open(filter_file_path, 'r') as filter_file_handle:
    filter_file_reader = csv.reader(filter_file_handle, delimiter = '\t')
    for row in filter_file_reader:
        if row[0] == 'events':
            keep_indices = [int(ele) for ele in row[1:]]

print 'loaded keep indices (', len(keep_indices), 'events total):\n', keep_indices
filtered_oi_events = [oi_events[i] for i in range(len(oi_events)) if i in keep_indices]
filtered_ellipsess = [ellipsess[i] for i in range(len(ellipsess)) if i in keep_indices]

In [ ]:
manual_remove_indices = []
oi_events = [filtered_oi_events[i] for i in range(len(filtered_oi_events)) if i not in manual_remove_indices]
ellipsess = [filtered_ellipsess[i] for i in range(len(filtered_ellipsess)) if i not in manual_remove_indices]

print 'Manually filtered', len(manual_remove_indices), 'events'

## Create plot

In [ ]:
def plot_ellipse(ellipse):
    
    x_center = ellipse[1]
    y_center = ellipse[2]
    a = ellipse[3]
    b = ellipse[4]
    theta = ellipse[5]
    
    
    # Plot ellipse perimiter
    xs = []
    ys = []
    for i in range(100):
        angle = i*2*np.pi/99.
        x_ellipse = a*np.cos(angle)
        y_ellipse = b*np.sin(angle)
        x = x_center + np.cos(theta)*x_ellipse + np.sin(theta)*y_ellipse
        y = y_center + np.sin(theta)*x_ellipse - np.cos(theta)*y_ellipse
        
        xs.append(x)
        ys.append(y)
        
    plt.plot(xs, ys, lw = .5, ls = ':', c = 'white')
    
    
    # Plot ellipse axes
    plt.plot([x_center, x_center + a], [y_center, y_center], lw = .5, c = 'white')
    plt.plot([x_center, x_center], [y_center, y_center + b], lw = .5, c = 'white')
    
    
    # Plot ellipse text
    yoffset = 5
    a_um = oi_stage.pixels_to_meters(a)
    b_um = oi_stage.pixels_to_meters(b)
    
    
    
    plt.text(0, 1.0,'a = ' + str(round(a_um, 2)) + r' $\mu$m', transform = plt.gca().transAxes, ha = 'left', va = 'top', color = 'white', fontweight = 'bold')    
    plt.text(0, 0.9,'b = ' + str(round(b_um, 2)) + r' $\mu$m', transform = plt.gca().transAxes, ha = 'left', va = 'top', color = 'white', fontweight = 'bold')    
    plt.text(0, 0.8,'a/b = ' + str(round(a/b, 2)), transform = plt.gca().transAxes, ha = 'left', va = 'top', color = 'white', fontweight = 'bold')    
    plt.text(0, 0.7,r'$\theta$ = ' + str(round(theta, 2)), transform = plt.gca().transAxes, ha = 'left', va = 'top', color = 'white', fontweight = 'bold')    
    


In [ ]:
# Loop over events
for i, oi_event in enumerate(oi_events):
    print i
    
    try:
        ellipses = ellipsess[i]


        # Create figure
        num_rows = 3
        num_cols = 3
        num_figs = num_rows*num_cols
        
        increment = 1.*len(ellipses)/num_figs
        
        
        fig, axes = plt.subplots(num_rows, num_cols, figsize = (8,8))
        plt.suptitle('event #' + str(i), y = 1.)
        for j in range(num_rows*num_cols):
            index = int(j*increment)
            ellipse = ellipses[index]
            detection_index = int(ellipse[0])
            frame_index = oi_event._detections[detection_index]._tf
            
            
            plt.sca(axes[j/num_cols, j%num_cols])
            plt.imshow(oi_vid.get_frame(frame_index), cmap = 'gray', origin = 'lower')
            plot_ellipse(ellipse)
            
            
            x_center = ellipse[1]
            y_center = ellipse[2]
            
            plt.xlim(x_center - 30, x_center + 30)
            plt.ylim(y_center - 30, y_center + 30)
            
            plt.xticks([])
            plt.yticks([])
            
        fig.tight_layout()
            
            
            
            
            

        plt.savefig(save_base_path + 'ellipse_fits/' + str(i) + '.png', dpi = 100)
        plt.close()
        plt.show()
        
    except:
        print '\tfailed on ', i
    

